In [2]:
!hostname

jupyter-g163771


### MÅL Å GJENSKAPE ETL JOBB (DT_1_DT_AGG_STONAD_MOTTAKER_MND)

In [3]:
import pandas as pd
import cx_Oracle as cx_Oracle
import time
import os
import json
from dataverk_vault import api as vault_api

vault_api.set_secrets_as_envs()
response = os.environ["dvh"]
config = json.loads(response)

In [10]:
month = 202110

In [11]:
sql = """
CREATE PRIVATE TEMPORARY TABLE ora$ptt_CTE AS
SELECT
FK_DIM_ALDER
,FK_DIM_KJONN
,FK_DIM_F_STONAD_OMRAADE
,FK_DIM_SERVICE_GRUPPE
,FK_DIM_ARBEIDSSOKERSTAT_AINFO AS FK_DIM_ARBEIDSSOKERSTATUS
,FK_DIM_HOVEDMAAL
,FK_DIM_TID_MND
,FK_DIM_ORGANISASJON
,FK_DIM_GEOGRAFI_BOSTED
,FK_DIM_VARIGHET_MOTTAKER AS FK_DIM_VARIGHET
,FK_DIM_OVERGANG_TILFELLE
,SUM(MOTTAKER_TILGANG_FLAGG) AS MOTTAKER_TILGANG_ANTALL
,SUM(MOTTAKER_FLAGG) AS MOTTAKER_ANTALL
,SUM(MOTTAKER_AVGANG_FLAGG) AS MOTTAKER_AVGANG_ANTALL
,SUM(VEDTAK_14A_FLAGG) AS VEDTAK_14A_ANTALL
,SUM(AKTUELL_FOR_TILTAK_FLAGG) AS AKTUELL_FOR_TILTAK_ANTALL
,SUM(DELTAKER_TILTAK_FLAGG)AS DELTAKER_TILTAK_ANTALL
,SUM(GODKJENT_PLAN_FLAGG) AS GODKJENT_PLAN_ANTALL
,SUM((CASE WHEN AKTUELL_FOR_TILTAK_FLAGG = 1 AND GODKJENT_PLAN_FLAGG = 1 THEN 1 ELSE 0 END)) AS GODKJ_PLAN_M_TILTAK_ANTALL
,SUM(AKT_TILT_DELTAR_TILT_FLAGG) AS AKT_TILT_DELTAR_TILT_ANTALL
,SUM(VARIGHET_MNDR_ANTALL) AS VARIGHET_MNDR_ANTALL
,SYSDATE AS LASTET_DATO
,KILDESYSTEM
FROM 
DT_P.FAK_STONAD_MOTTAKER
WHERE 
TO_NUMBER(SUBSTR(DT_P.FAK_STONAD_MOTTAKER.FK_DIM_TID_MND,1 ,6))= {}
GROUP BY
FK_DIM_ALDER
,FK_DIM_KJONN
,FK_DIM_F_STONAD_OMRAADE
,FK_DIM_SERVICE_GRUPPE
,FK_DIM_ARBEIDSSOKERSTAT_AINFO
,FK_DIM_HOVEDMAAL
,FK_DIM_TID_MND
,FK_DIM_ORGANISASJON
,FK_DIM_GEOGRAFI_BOSTED
,FK_DIM_VARIGHET_MOTTAKER
,FK_DIM_OVERGANG_TILFELLE
,SYSDATE
,KILDESYSTEM
"""

sql_1= """
MERGE INTO DT_P.AGG_STONAD_MOTTAKER_MND_DRP202212 X
USING (SELECT
FK_DIM_ALDER
,FK_DIM_KJONN
,FK_DIM_F_STONAD_OMRAADE
,FK_DIM_SERVICE_GRUPPE
,FK_DIM_ARBEIDSSOKERSTATUS
,FK_DIM_HOVEDMAAL
,FK_DIM_TID_MND
,FK_DIM_ORGANISASJON
,FK_DIM_GEOGRAFI_BOSTED
,FK_DIM_VARIGHET
,FK_DIM_OVERGANG_TILFELLE
,MOTTAKER_TILGANG_ANTALL
,MOTTAKER_ANTALL
,MOTTAKER_AVGANG_ANTALL
,VEDTAK_14A_ANTALL
,AKTUELL_FOR_TILTAK_ANTALL
,DELTAKER_TILTAK_ANTALL
,GODKJENT_PLAN_ANTALL
,GODKJ_PLAN_M_TILTAK_ANTALL
,AKT_TILT_DELTAR_TILT_ANTALL
,VARIGHET_MNDR_ANTALL
,LASTET_DATO
,KILDESYSTEM
FROM ora$ptt_CTE) Y ON     
(X.FK_DIM_ALDER = Y.FK_DIM_ALDER AND
X.FK_DIM_KJONN = Y.FK_DIM_KJONN AND
X.FK_DIM_F_STONAD_OMRAADE = Y.FK_DIM_F_STONAD_OMRAADE AND
X.FK_DIM_SERVICE_GRUPPE = Y.FK_DIM_SERVICE_GRUPPE AND
X.FK_DIM_ARBEIDSSOKERSTATUS = Y.FK_DIM_ARBEIDSSOKERSTATUS AND
X.FK_DIM_HOVEDMAAL = Y.FK_DIM_HOVEDMAAL AND
X.FK_DIM_TID_MND = Y.FK_DIM_TID_MND AND
X.FK_DIM_ORGANISASJON = Y.FK_DIM_ORGANISASJON AND
X.FK_DIM_GEOGRAFI_BOSTED = Y.FK_DIM_GEOGRAFI_BOSTED AND
X.FK_DIM_VARIGHET = Y.FK_DIM_VARIGHET AND
X.FK_DIM_OVERGANG_TILFELLE = Y.FK_DIM_OVERGANG_TILFELLE AND
X.KILDESYSTEM = Y.KILDESYSTEM)
WHEN NOT MATCHED THEN INSERT(
X.FK_DIM_ALDER
,X.FK_DIM_KJONN
,X.FK_DIM_F_STONAD_OMRAADE
,X.FK_DIM_SERVICE_GRUPPE
,X.FK_DIM_ARBEIDSSOKERSTATUS
,X.FK_DIM_HOVEDMAAL
,X.FK_DIM_TID_MND
,X.FK_DIM_ORGANISASJON
,X.FK_DIM_GEOGRAFI_BOSTED
,X.FK_DIM_VARIGHET
,X.FK_DIM_OVERGANG_TILFELLE
,X.MOTTAKER_TILGANG_ANTALL
,X.MOTTAKER_ANTALL
,X.MOTTAKER_AVGANG_ANTALL
,X.VEDTAK_14A_ANTALL
,X.AKTUELL_FOR_TILTAK_ANTALL
,X.DELTAKER_TILTAK_ANTALL
,X.GODKJENT_PLAN_ANTALL
,X.GODKJ_PLAN_M_TILTAK_ANTALL
,X.AKT_TILT_DELTAR_TILT_ANTALL
,X.VARIGHET_MNDR_ANTALL
,X.LASTET_DATO
,X.KILDESYSTEM)
VALUES
(Y.FK_DIM_ALDER
,Y.FK_DIM_KJONN
,Y.FK_DIM_F_STONAD_OMRAADE
,Y.FK_DIM_SERVICE_GRUPPE
,Y.FK_DIM_ARBEIDSSOKERSTATUS
,Y.FK_DIM_HOVEDMAAL
,Y.FK_DIM_TID_MND
,Y.FK_DIM_ORGANISASJON
,Y.FK_DIM_GEOGRAFI_BOSTED
,Y.FK_DIM_VARIGHET
,Y.FK_DIM_OVERGANG_TILFELLE
,Y.MOTTAKER_TILGANG_ANTALL
,Y.MOTTAKER_ANTALL
,Y.MOTTAKER_AVGANG_ANTALL
,Y.VEDTAK_14A_ANTALL
,Y.AKTUELL_FOR_TILTAK_ANTALL
,Y.DELTAKER_TILTAK_ANTALL
,Y.GODKJENT_PLAN_ANTALL
,Y.GODKJ_PLAN_M_TILTAK_ANTALL
,Y.AKT_TILT_DELTAR_TILT_ANTALL
,Y.VARIGHET_MNDR_ANTALL
,Y.LASTET_DATO
,Y.KILDESYSTEM)
WHEN MATCHED THEN UPDATE SET
X.MOTTAKER_TILGANG_ANTALL = Y.MOTTAKER_TILGANG_ANTALL
,X.MOTTAKER_ANTALL = Y.MOTTAKER_ANTALL
,X.MOTTAKER_AVGANG_ANTALL = Y.MOTTAKER_AVGANG_ANTALL
,X.VEDTAK_14A_ANTALL = Y.VEDTAK_14A_ANTALL
,X.AKTUELL_FOR_TILTAK_ANTALL = Y.AKTUELL_FOR_TILTAK_ANTALL
,X.DELTAKER_TILTAK_ANTALL = Y.DELTAKER_TILTAK_ANTALL
,X.GODKJENT_PLAN_ANTALL = Y.GODKJENT_PLAN_ANTALL
,X.GODKJ_PLAN_M_TILTAK_ANTALL = Y.GODKJ_PLAN_M_TILTAK_ANTALL
,X.AKT_TILT_DELTAR_TILT_ANTALL = Y.AKT_TILT_DELTAR_TILT_ANTALL
,X.VARIGHET_MNDR_ANTALL = Y.VARIGHET_MNDR_ANTALL
,X.LASTET_DATO = Y.LASTET_DATO
"""

sql_2 = "COMMIT"

In [12]:
try:
    con = cx_Oracle.connect(**config["U"])
except:
    print("Oracle: Unable to connect")

cur = con.cursor()

In [13]:
try:
    cur.execute(sql.format(month))
    cur.execute(sql_1)
except cx_Oracle.DatabaseError as exc:
    error, = exc.args
    print("Oracle-Error-Code:", error.code)
    print("Oracle-Error-Message:", error.message)
    print("Oracle-Error-Offset:", error.offset)
    print("Oracle-Error-Context:", error.context)
    print("Oracle-Error-isrecoverable:", error.isrecoverable)
finally:
    cur.execute(sql_2)
    if cur:
        cur.close()
    if con:
        con.close()